In [1]:
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np

# Load images from a pickle file
with open("stl_test.pickle", "rb") as tr_images_file:
    test = pickle.load(tr_images_file)
with open("stl_train.pickle", "rb") as tr_images_file:
    train = pickle.load(tr_images_file)
with open("stl_val.pickle", "rb") as tr_images_file:
    new = pickle.load(tr_images_file)
with open("stl_deg.pickle", "rb") as tr_images_file:
    deg = pickle.load(tr_images_file)




so far so good...


In [3]:

def unpk(data):

    x=[]
    y=[]
    c=0
    tot=[]
    for pair in data:
        try:
            img=np.array(pair[0]).reshape((96,96,3))
            x.append(img)
            y.append(pair[1])
        except Exception as e:
            c=c+1
    
    print("exceptions")
    print(c)
    return x, y

x_train,y_trai=unpk(train)
x_test,y_tes=unpk(test)
x_new, y_ne=unpk(new)
x_deg, y_deg=unpk(deg)
c2=0


eccezioni
0
eccezioni
0
eccezioni
0
eccezioni
0


In [4]:

y_deg=[]
y_train=[]
y_test=[]
y_new=[]

#this part is to reasses the labels so that label 9 is drift
for i in range(len(x_deg)):
    y_deg.append(9)

for i in range(len(y_trai)):
    if y_trai[i]==9:
        y_train.append(3)
    else:
        y_train.append(y_trai[i])

for i in range(len(y_tes)):
    if y_tes[i]==9:
        y_test.append(3)
    else:
        y_test.append(y_tes[i])

for i in range(len(y_ne)):
    if y_ne[i]==9:
        y_new.append(3)
    else:
        y_new.append(y_ne[i])



In [5]:
new_train_x=x_train.copy()
new_train_y=y_train.copy()

new_x_test=[]
new_y_test=[]

for i in range(len(x_test)):
        new_x_test.append(x_test[i])
        new_y_test.append(y_test[i])
    

new_train_x=np.array(new_train_x)
new_test_x=np.array(new_x_test)
new_train_y=np.array(new_train_y)
new_test_y=np.array(new_y_test)
x_new=np.array(x_new)
x_deg=np.array(x_deg)
y_new=np.array(y_new)
y_deg=np.array(y_deg)

In [6]:
print(len(new_train_x))
print(len(new_test_x))
print(len(x_new))
print(len(x_deg))
print(type(new_train_x))
print(type(new_train_x))
print(type(x_new))
print(type(x_deg))

5850
2925
2925
1300
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [8]:
from tensorflow.keras.utils import to_categorical

print(new_train_y[0])

cat_lab_train=to_categorical(new_train_y)
cat_lab_test=to_categorical(new_test_y)
cat_lab_new=to_categorical(y_new)
cat_lab_deg=to_categorical(y_deg)

print(cat_lab_train[0])

2023-11-24 15:17:43.524895: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 15:17:43.560251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-24 15:17:43.560287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-24 15:17:43.561984: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-24 15:17:43.568865: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-24 15:17:43.569561: I tensorflow/core/platform/cpu_feature_guard.cc:1

0
[1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense


base_model=VGG16(weights='imagenet', include_top=False, input_shape=(96,96,3))


for layer in base_model.layers[:]:
        layer.trainable = False
        
model=Sequential()
model.add(base_model)
model.add(Flatten(name="fl"))
model.add(Dense(256, activation="relu", name="fc"))
model.add(Dense(9, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"])

model.fit(new_train_x, cat_lab_train, batch_size=50, epochs=10, validation_split=0)


#model_e= Model(inputs=model.input, outputs=model.get_layer('fc').output)
model_e=base_model

Epoch 1/10
117/117 [==============================] - 23s 191ms/step - loss: 10.2187 - acc: 0.6347
Epoch 2/10
117/117 [==============================] - 21s 180ms/step - loss: 0.5732 - acc: 0.8103
Epoch 3/10
117/117 [==============================] - 21s 180ms/step - loss: 0.3982 - acc: 0.8607
Epoch 4/10
117/117 [==============================] - 21s 180ms/step - loss: 0.2673 - acc: 0.9039
Epoch 5/10
117/117 [==============================] - 21s 181ms/step - loss: 0.2024 - acc: 0.9270
Epoch 6/10
117/117 [==============================] - 21s 183ms/step - loss: 0.1597 - acc: 0.9443
Epoch 7/10
117/117 [==============================] - 21s 183ms/step - loss: 0.1216 - acc: 0.9561
Epoch 8/10
117/117 [==============================] - 21s 182ms/step - loss: 0.1057 - acc: 0.9617
Epoch 9/10
117/117 [==============================] - 21s 182ms/step - loss: 0.0865 - acc: 0.9694
Epoch 10/10
117/117 [==============================] - 21s 181ms/step - loss: 0.0618 - acc: 0.9762


In [10]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score


pred=model.predict(new_test_x)

print(confusion_matrix(new_test_y, pred.argmax(1)))

# Print classification report
print("Classification Report:")
print(classification_report(new_test_y, pred.argmax(1)))


92/92 [==============================] - 12s 125ms/step
[[223   0  43  13  15   2  25   0   4]
 [  1 303   0   1   2  10   1   4   3]
 [ 35   0 215  19  14   0  22   0  20]
 [ 14   0   9 269  10   1  11   0  11]
 [ 13   0   7   5 277   3  20   0   0]
 [  1   6   0   2   6 289   0  18   3]
 [ 19   0  20   6  16   0 258   0   6]
 [  1   3   0   0   0  21   0 297   3]
 [  7   0  17  37   4   3   9   4 244]]
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.69      0.70       325
           1       0.97      0.93      0.95       325
           2       0.69      0.66      0.68       325
           3       0.76      0.83      0.79       325
           4       0.81      0.85      0.83       325
           5       0.88      0.89      0.88       325
           6       0.75      0.79      0.77       325
           7       0.92      0.91      0.92       325
           8       0.83      0.75      0.79       325

    accuracy               

In [12]:

def embedding_creation(X, y):
    predictions=model.predict(X)
    #conv_embedding=base_model.predict(X)
    embedding=model_e.predict(X)

    emb_list=[]
    
    prediction_list=[]

    for i in range(len(X)):
        emb_list.append(embedding[i])
        #conv_emb_list.append(conv_embedding[i])
        prediction_list.append(predictions[i])

    emb_list=np.array(emb_list)
    #conv_emb_list=np.array(conv_emb_list)

    prediction_list=np.array(prediction_list)
    prediction_list=np.argmax(prediction_list, axis=1)
    

    return prediction_list, emb_list,  y
    
def save_embeddings(embedding, predicted_label, true_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("E", data=embedding, compression="gzip")
    fp.create_dataset("Y_predicted", data=predicted_label, compression="gzip") 
    fp.create_dataset("Y_original", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")


In [13]:
import h5py


print("training embedding extraction...")
prediction_training, embedding_training, true_label_trainining=embedding_creation(new_train_x, new_train_y)
print("test embedding creation...")
prediction_test, embedding_test,  true_label_test=embedding_creation(new_test_x, new_test_y)
print("new embedding creation...")
prediction_new, embedding_new, true_label_new=embedding_creation(x_new, y_new)
print("drifted data embedding creation...")
#el=False
prediction_deg, embedding_deg, true_label_deg=embedding_creation(x_deg, y_deg)

print("saving the embeddings...")

save_embeddings(embedding_training, prediction_training, true_label_trainining, "Conv_VGG-16_Stl_train.hdf5" )
save_embeddings(embedding_test, prediction_test, true_label_test, "Conv_VGG-16_Stl_test.hdf5" )
save_embeddings(embedding_new, prediction_new, true_label_new, "Conv_VGG-16_Stl_new.hdf5" )
save_embeddings(embedding_deg, prediction_deg, true_label_deg, "Conv_VGG-16_Stl_deg.hdf5" )



training embedding extraction...
183/183 [==============================] - 23s 126ms/step
test embedding creation...
92/92 [==============================] - 12s 128ms/step
new embedding creation...
92/92 [==============================] - 12s 126ms/step
degerated data embedding creation...
41/41 [==============================] - 5s 125ms/step
saving the embeddings...
embeddings saved
embeddings saved
embeddings saved
embeddings saved
